In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



In [5]:
import pandas as pd
df = pd.read_csv("Unemployment_Rate_upto_11_2020.csv")  # adjust filename
df.head()
print(df.columns.tolist())
df.info()


['Region', ' Date', ' Frequency', ' Estimated Unemployment Rate (%)', ' Estimated Employed', ' Estimated Labour Participation Rate (%)', 'Region.1', 'longitude', 'latitude']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 267 entries, 0 to 266
Data columns (total 9 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Region                                    267 non-null    object 
 1    Date                                     267 non-null    object 
 2    Frequency                                267 non-null    object 
 3    Estimated Unemployment Rate (%)          267 non-null    float64
 4    Estimated Employed                       267 non-null    int64  
 5    Estimated Labour Participation Rate (%)  267 non-null    float64
 6   Region.1                                  267 non-null    object 
 7   longitude                                 267 non-null    float64
 8   latitud

In [8]:
print(df.columns())


TypeError: 'Index' object is not callable

In [6]:
df.columns = [c.strip().lower().replace(' ', '_').replace('(', '').replace(')', '') for c in df.columns]

# typical names: state, date, frequency, estimated_unemployment_rate_percent

df = df.rename(columns=lambda x: x.replace('%','pct'))

# parse date (some Kaggle files use 'yyyy-mm' or 'dd-mm-yyyy')

df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=False)
df = df.sort_values(['state','date']).reset_index(drop=True)


C:\Users\User\AppData\Local\Temp\ipykernel_6820\727067756.py:9: UserWarning: Parsing dates in  %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['date'] = pd.to_datetime(df['date'], errors='coerce', dayfirst=False)


KeyError: 'state'